In [1]:
import io

import netaddr

import pandas as pd

import aiohttp

from rpki_analysis.delegated_stats import read_delegated_extended_stats

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        df = read_delegated_extended_stats(io.StringIO(await resp.text()))


In [ ]:
list(df.sample(1).resource)[0]

In [ ]:
# Calculate cover of VRPs
def join_resources(df: pd.DataFrame) -> pd.DataFrame:
    """Join the resources of a group"""
    return pd.DataFrame([{
        "resources": netaddr.IPSet(df[df.afi != "asn"].resource),
    }])


In [ ]:
df_resources = df.groupby(['opaque_id', 'country', 'afi', 'rir']).apply(join_resources)

In [ ]:
df_resources['resource_size'] = df_resources.resources.map(lambda x: x.size)

In [ ]:
df_resources[['resource_size']].sample(1)


In [ ]:
def majority_size(df: pd.DataFrame) -> float:
    sizes = list(df.resource_size)
    sizes.sort(reverse=True)
    total_sizes = sum(sizes)
    if total_sizes > 0:
        return pd.DataFrame([{"covered_fraction": sizes[0]/sum(sizes), "total_resources": total_sizes }])
    return pd.DataFrame([{"covered_fraction": 0, "total_resources": 0 }])
    
df_fraction_majority = df_resources[['resource_size']].reset_index().groupby(['opaque_id', 'afi', 'rir']).apply(majority_size)

In [ ]:
df_fraction_majority

In [ ]:
df_ip = df_fraction_majority.reset_index()[df_fraction_majority.reset_index().afi != "asn"]

In [ ]:
df_ip[(df_ip.covered_fraction < 1.0) & (df_ip.rir == "ripencc")].sort_values(["total_resources"])

In [ ]:
df[df.opaque_id == "b06b370c-da4c-429b-ba92-322c81c9662d"].country.unique()

Check how much of the space in the country is by RIPE NCC

In [ ]:
def rir_fraction(df: pd.DataFrame, rir="ripencc") -> float:
    rir_total = sum(df[df.rir == rir].resource_size)
    total = sum(df.resource_size)
    
    if total > 0:
        return pd.DataFrame([{rir: rir_total/total, f"{rir}_resources": rir_total, "total_resources": total }])
    return pd.DataFrame([{rir: 0, f"{rir}_resources": 0, "total_resources": 0 }])
    
df_fraction_majority = df_resources[['resource_size']] \
    .reset_index() \
    .groupby(['afi', 'country']) \
    .apply(rir_fraction) \
    .reset_index()

In [ ]:
df_fraction_majority[df_fraction_majority.afi != "asn"]

In [ ]:
import os

df_fraction_majority[df_fraction_majority.afi != "asn"].to_excel(os.path.expanduser("~/Desktop/fraction-non-ripe-ncc-by-afi-country.xlsx")) 